In [ ]:
import sys
import os
current_dir = os.getcwd()
sys.path.append(current_dir)  # 루트 디렉터리 경로 추가  # .py
sys.path.append(os.path.dirname(current_dir))  # 상위 디렉터리 경로 추가  # .ipynb

from src.crawling.nst_item_search import *
import pandas as pd

In [ ]:
# 네이버 트렌드 데이터 수집 진행


# 게임 리스트
game_name_list = "쿠키런킹덤, 던전앤파이터, 메이플스토리, 월드오브워크래프트, 블레이드앤소울, 마비노기, 리그오브레전드, 펜타스톰, 도타2, 브롤스타즈, 서든어택, 배틀그라운드, 오버워치, 발로란트, 클래시오브클랜, 클래시로얄, 스타크래프트, 피파온라인, 피파23, 캔디크러쉬사가, 마인크래프트, 로블록스, 카트라이더, 테일즈런너, 크레이지아케이드, 모여봐요동물의숲, 모두의마블, 포켓몬스터".split(', ')

# 1. URL 접속
driver = webdriver.Chrome()

url = "https://datalab.naver.com/"
driver.get(url)                 # URL로 이동

# 기본 옵션 설정

# 범위 설정 (월 기준)
select_dropdown_option(driver, "timeDimensionTitle", "//li[contains(@class, '_item_month')]/a")
# # 범위 설정 (주 기준)
# select_dropdown_option(driver, "timeDimensionTitle", "//li[contains(@class, '_item_week')]/a")

# 시작 연월 설정
select_dropdown_option(driver, "startYear", "//div[@id='startYearDiv']//li[contains(@class, '_item_2023')]/a")
select_dropdown_option(driver, "startMonth", "//div[@id='startMonthDiv']//li[contains(@class, '_item_01')]/a")

# 종료 연월 설정
select_dropdown_option(driver, "endYear", "//div[@id='endYearDiv']//li[contains(@class, '_item_2023')]/a")
select_dropdown_option(driver, "endMonth", "//div[@id='endMonthDiv']//li[contains(@class, '_item_12')]/a")

# 연령 체크박스 정보
age_info = {
    '06_12': "item_age_1",
    '13_18': "item_age_2",
    '19_24': "item_age_3",
    '25_29': "item_age_4",
    '30_34': "item_age_5",
    '35_39': "item_age_6"
}

trend_data = []
for age in age_info:
    for uncheck in age_info.values():  # 체크박스 초기화
        toggle_checkbox(driver, uncheck, check=False)
    toggle_checkbox(driver, age_info[age], check=True)
    # for gt in game_trend_list:
    #     input_keywords(driver, gt)  # 키워드 입력
    for g in game_name_list[:2]:
        if (age, g) in [(t['age'], t['name']) for t in trend_data]:
            continue
        input_keyword(driver, baseline_keyword="게임", input_keyword=g+",게임")  # 키워드 입력
        search_btn = driver.find_element(
            By.CSS_SELECTOR,
            "#content > div._search_trend_wrapper > div.keyword_trend > div > div > form > fieldset > a, #content > div.section_keyword > div.keyword_trend > div.section_step > div.com_box_inner > form > fieldset > a"
        )
        search_btn.click()  # 조회버튼 클릭
        print(age, g, end="/")
        # print(gt)
        time.sleep(2)
        trend_score = extract_relative_trend_score(driver)
        print(trend_score)
        # for g, s in zip(gt, trend_score):
        trend_data.append({
            "age": age,
            "name": g,
            "score": trend_score
        })


In [ ]:
df_trend_data = pd.DataFrame(trend_data)
df_trend_data['score'] = df_trend_data['score'] - 1
# df_trend_data.to_csv(data_path + "game_trend.csv", index=False)
df_trend_data

### 데이터 분석 및 시각화


In [ ]:
# df_trend_data = pd.read_csv(data_path + "game_trend.csv")
# # df_trend_data.loc[df_trend_data["age"] == "_12", "age"] = "06_12"
df_trend_data.sort_values(["name", "age"])